# Introduction
Welcome to this kernel about the Brackish dataset from AAU.
In this kernel we will come by the following steps:

* Loading annotations using Pandas.
* Specifying video and frame of interest.
* Extracting relevant annotations for video and frame.
* How to plot the annotated frame


# Loading annotations using Pandas
The annotation files will be loaded using pandas, since pandas is a great library for data analsys and provide an easy method for loading comma separated file (CSV), however, in this case the files are seperated by semicolons.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2 # For loading the videofile and plotting bounding boxes
import os # For loading datafiles
from matplotlib import pyplot as plt # For plotting the annotations

# Input data files are available in the "../input/" directory.
# Loading data, the data is semicolon seperated.

Valid = pd.read_csv("../input/brackish-dataset/annotations/annotations_AAU/valid.csv", sep=";")
Train = pd.read_csv("../input/brackish-dataset/annotations/annotations_AAU/train.csv", sep=";")
Test = pd.read_csv("../input/brackish-dataset/annotations/annotations_AAU/test.csv", sep=";")


# Specifying video and frame of interest

The only thing needed to be specified, is which videofile to use, and the corresponding frame number.
It is possible to plot the annotation for all the frames in the video, by iterating all frame in a for loop,
but in the kernel, we are only allowed to plot single frames and thereby we have chosen to allow for choosing a specific frame instead.

In [ ]:
# Specify which video and frame to plot annotations for
Video_File = "../input/brackish-dataset/dataset/videos/jellyfish/2019-03-20_15-15-12to2019-03-20_15-15-19_1.avi"
Frame_Num = 52

# Get basename, since the annotation is sorted by the filename
Filename = os.path.basename(Video_File).replace(".avi","")

# Extracting relevant annotations for video
One method do reduce the complexity of sorting out which annotating is important for the given videofile, is to get pandas to sort out annotations where the column ***"Filename"*** contains the video's filename.
Then a new array is created consisting of all the annotations for the given videofile. By doing so, we do not have to browse all the annotations when we change frame number, but only needs to browse the annotations for the given video.
This is indeed relavent, when you want to plot the annotations for all frame in the video.

In [ ]:
### Extracting all annotations for the video file using Pandas

# All annotations where the column filename contains the video filename.
AnnoV = Valid[Valid["Filename"].str.contains(Filename)]
AnnoT = Test[Test["Filename"].str.contains(Filename)]
AnnoTr = Train[Train["Filename"].str.contains(Filename)]

# Append annotations from validation, training and test set into one single array named Anno.
Anno = AnnoV
Anno = Anno.append(AnnoT)
Anno = Anno.append(AnnoTr)

# Extracting relevant annotations for frame
Again, here we use pandas to sort out the annotations, which contains the frame number in the ***"Filename"*** column.

In [ ]:
### Script for extracting all annotations for the given frame using Pandas

Frame_Anno = Anno[Anno["Filename"].str.contains("%04d" % Frame_Num)]  # "%04d" is for making 1 = 0001 to fit annotations.

# Drawing bounding boxes and plotting frame
In this script we load the videofile, and iterate through the frames one by one, and store them in a list called ***Images***. The frames are resized to the (960, 540) pixel resolution in order to fit the resolution at which the annotations are made.
Afterwards, we iterate through the array containing annotations for the given frame, drawing bounding boxes on the frame.
Lastly, we plot the frame with the annotations drawn.

In [ ]:
### Script for loading video and drawing bounding boxes
cap = cv2.VideoCapture(Video_File)

# Create a empty list, in which all the images from the video will be stored 
Images = []

while(True):
    ret, frame = cap.read()
    if ret == True:
        # Resizing the image to fit the resolution of the annotations (960 x 540) pixels.
        Images.append(cv2.resize(frame, (960, 540)))
    else:
        break

for idx, row in Frame_Anno.iterrows():
    
    # Extracting position of bounding boxes
    UX, UY, LX, LY = row["Upper left corner X"], row["Upper left corner Y"], row["Lower right corner X"], row["Lower right corner Y"]
    cv2.rectangle(Images[Frame_Num], (UX,UY), (LX, LY), (0, 255, 0), 5) # Drawing bounding boxes

# Plot the annotated image
plt.imshow(Images[Frame_Num])


# Annotate all frames in a video
This script does annotate all the frames in a given video, by iterating through the annotation file called ***Anno***. In this example we just choose a random image (55) and plot this image, however, one could be interest in see the entire video. And this can be carried out by iterating the ***Images*** list in a loop.

In [ ]:
### Script for annotating all frames in a video.
for idx, row in Anno.iterrows():
    
    # Extracting the frame number for the annotation.
    # Kinda hardcode though, but it works.
    framenum = int(row["Filename"].replace(Filename + "-","").replace(".png",""))
    
    # Again extracting needed information about the bounding box.
    UX, UY, LX, LY = row["Upper left corner X"], row["Upper left corner Y"], row["Lower right corner X"], row["Lower right corner Y"]
    cv2.rectangle(Images[framenum], (UX,UY), (LX, LY), (0, 255, 0), 5) # Drawing bounding boxes

# Plot the annotated frames, can be inserted into a for loop to show all images.
plt.imshow(Images[55])
    

# Ending
This was all for now, more will come later on.